# Generate trades for multi identifiers using dedicated service - Python

### Overview
Retrieving trades over a look back period is a valuable indicator to highlight market activity. It can be correlated with a set of technical indicators to better analyze market movements and build a decision making parameter.

This sample is designed to request trades and illustrate for many assets at once.

This sample demonstrates how to request and plot from a dedicated data service **on-demand** trades over the time.

### Inputs/outputs
Trades extraction sample requires instrument's identifier, date time intervals as per inputs and returns the occured trades information.

### Services used
This sample uses *gRPC requests* in order to retrieve trades information from the hosted service. The queried endpoint in this script are:
* *TickTradesService*: to directly retrieve trades data from the server.

### Modules required
1. Systemathics:
    * *systemathics.apis.services.tick.v1*
    * *systemathics.apis.type.shared.v1*
    * *google.type*
2. Open source:
    * *googleapis-common-protos*
    * *protobuf*
    * *grpcio*
    * *pandas*
    * *plotly* and *matpotlib* as per display packages
    
***

# Run Trades sample for multiple identifiers

### Step 1: Install packages and import them

In [ ]:
pip install googleapis-common-protos protobuf grpcio pandas matplotlib plotly

In [ ]:
pip install systemathics.apis --pre

In [ ]:
import os
import grpc
import pandas as pd
from datetime import datetime
import google.type.date_pb2 as date
import google.type.timeofday_pb2 as timeofday
import google.type.dayofweek_pb2 as dayofweek
import google.protobuf.duration_pb2 as duration
import systemathics.apis.type.shared.v1.identifier_pb2 as identifier
import systemathics.apis.type.shared.v1.constraints_pb2 as constraints
import systemathics.apis.type.shared.v1.date_interval_pb2 as dateinterval
import systemathics.apis.type.shared.v1.time_interval_pb2 as timeinterval
import systemathics.apis.services.tick.v1.tick_trades_pb2 as tick_trades
import systemathics.apis.services.tick.v1.tick_trades_pb2_grpc as tick_trades_service
import systemathics.apis.helpers.token_helpers as token_helpers
import systemathics.apis.helpers.channel_helpers as channel_helpers

### Step 2: Retrieve authentication token
The following code snippet sends authentication request and print token to console output in order to process the upcomming *gRPC queries*.

In [ ]:
token = token_helpers.get_token()
display(token)

### Step 3: Retrieve data
To request *tick trades* service, we need to specify:
* Instrument identifiers
* Time period selection: select start and end dates
* Tick trades request parameters

#### 3.1 Instrument selection

In [ ]:
# generate the tick trades request for the different assets
aapl_bats = identifier.Identifier(exchange = "BATS", ticker = "AAPL")
aapl_xngs = identifier.Identifier(exchange = "XNGS", ticker = "AAPL")

#### 3.2 Time period delimitation

In [ ]:
# create time intervals (we are using Google date format)
date_interval = dateinterval.DateInterval(
    start_date = date.Date(year = 2021, month = 1, day = 5), 
    end_date = date.Date(year = 2021, month = 1, day = 5)
)

# build the market data request time interval (we are using Google time format)
# UTC time zone
time_interval = timeinterval.TimeInterval(
    start_time = timeofday.TimeOfDay(hours = 12, minutes = 0, seconds = 3), 
    end_time = timeofday.TimeOfDay(hours = 12, minutes = 0, seconds = 10)
)

#### 3.3 Request creation
The following code snippet creates *gRPC client*, process request and returns the request reply:

In [ ]:
# generate constraints based on the previous time selection
my_constraints = constraints.Constraints(
    date_intervals = [date_interval],
    time_intervals = [time_interval],
)

In [ ]:
# generate the tick trades request with multiple identifiers
request = tick_trades.TickTradesRequest(
    identifiers = [aapl_bats,aapl_xngs],
    constraints = my_constraints
)

In [ ]:
try:
    # open a gRPC channel
    with channel_helpers.get_grpc_channel() as channel:
        
        # instantiate the tick trades service
        service = tick_trades_service.TickTradesServiceStub(channel)
        
        # process the tick trades request
        trades = []
        metadata = [('authorization', token)]
        for trade in service.TickTrades(request=request, metadata=metadata):
            if trade.data.time_stamp.seconds > 0:
                trades.append(trade)
            else:
                mappings = trade.mapping

except grpc.RpcError as e:
    display(e.code().name)
    display(e.details())

The following code snippet displays the total retrieved trades count:

In [ ]:
print('Total trades retrieved: {0}'.format(len(trades)))

In [ ]:
# Display how the mapping is organized
providers = [m.identifier.provider.value for m in mappings.values]
tickers = [m.identifier.ticker for m in mappings.values]
exchanges = [m.identifier.exchange for m in mappings.values]
maps = [m.value for m in mappings.values]

d = {'Providers': providers, 'Tickers': tickers, 'Exchanges': exchanges, 'Mappings': maps}

mappingsdf = pd.DataFrame(data=d)
mappingsdf

### Step 4: Visualize data

#### 4.1 Retrieve data
In the following code snippets, the reply is reprocessed in a data frame in order to visualize the results with ease:

In [ ]:
# small method to get a better display of the identifiers
def prettyprint(identifier):
  return identifier.provider.value + ">" + identifier.exchange + ">" + identifier.ticker

In [ ]:
from datetime import timedelta

# prepare the dataframe content
dates = [(datetime.fromtimestamp(t.data.time_stamp.seconds) + timedelta(microseconds = t.data.time_stamp.nanos/1000))  for t in trades]
prices = [t.data.trade.price for t in trades]
sizes = [t.data.trade.size for t in trades]
maps = [t.data.mapping for t in trades]
identifiers = [prettyprint(mappings.values[t.data.mapping].identifier) for t in trades]
#identifiers = [t.data.mapping for t in trades]

# create a pandas dataframe with: dates, eventsources, identifiers, trades prices and sizes
d = {'Date': dates, 'Mappings': maps, 'Identifier': identifiers, 'Price': prices, 'Size': sizes}
#d = {'Date': dates, 'Mappings': maps, 'Price': prices, 'Size': sizes}
df = pd.DataFrame(data=d)
pd.set_option('display.max_rows', None)

In [ ]:
df